[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/03_modelado/09_comparativa_cualitativa.ipynb)
[![View on GitHub](https://img.shields.io/badge/View_on-GitHub-black?logo=github)](https://github.com/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/03_modelado/09_comparativa_cualitativa.ipynb)

# Comparativa cualitativa de modelos

En los apartados anteriores observamos que las métricas globales (Top@k, MRR) mostraban diferencias poco concluyentes entre **Kneser–Ney (KN)** y las **redes neuronales recurrentes (LSTM)**.  
Con el fin de contrastar su **utilidad en situaciones musicales concretas**, este notebook presenta una **evaluación cualitativa ad hoc**.

Para ello definimos una *suite* de progresiones características en distintos contextos armónicos (mayor, menor, dominantes secundarios y sustitutos, casos de *cold-start* y rarezas).  
En cada caso comparamos las **sugerencias generadas por ambos modelos**, analizando hasta qué punto resultan plausibles desde el punto de vista musical.

Este enfoque complementa el análisis estadístico con una valoración más cercana a la práctica armónica real, y ayuda a justificar la elección del modelo final para el despliegue.

**Contenido del notebook**:
1. Carga del modelo LSTM preentrenado
2. Carga del modelo Kneyser-Ney preentrenado
3. Comparativa cualitativa de predicciones: KN vs LSTM
   - Progresiones en tonalidad mayor
   - Progresiones en tonalidad menor
   - Dominantes secundarios y sustitutos
   - Casos de *cold-start*
   - Rarezas


## 1) Carga del modelo lstm preentrenado

1.1. Importamos el modelo preentrenado desde GitHub.

In [ ]:
import urllib.request
import os

# Configuración para descargar el modelo desde GitHub
USER = "antoniotrapote"
REPO = "chord-prediction-tfm"
BRANCH = "main"
PATH_IN_REPO = "anexos/models/lstm_rs_v3/lstm_rs_v3.pt"
MODEL_URL = f"https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}/{PATH_IN_REPO}"

# Ruta local donde guardar el archivo
model_path = "/content/lstm_rs_v3.pt"

try:
    # Descargar el archivo del modelo desde GitHub
    print(f"Descargando modelo desde: {MODEL_URL}")
    urllib.request.urlretrieve(MODEL_URL, model_path)
    print(f"Modelo descargado exitosamente en: {model_path}")

    # Verificar que el archivo se descargó correctamente
    if os.path.exists(model_path):
        file_size = os.path.getsize(model_path)
        print(f"Tamaño del archivo: {file_size / (1024*1024):.2f} MB")
    else:
        print("Error: El archivo no se descargó correctamente")

except Exception as e:
    print(f"Error al descargar el modelo: {e}")

Descargando modelo desde: https://raw.githubusercontent.com/antoniotrapote/chord-prediction-tfm/main/anexos/models/lstm_rs_v3/lstm_rs_v3.pt
Modelo descargado exitosamente en: /content/lstm_rs_v3.pt
Tamaño del archivo: 5.95 MB


1.2. Recreamos la arquitectura del modelo y cargamos los pesos.

In [ ]:
import torch, torch.nn as nn

class ChordLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout, tie_weights=False):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers,
                           batch_first=True, dropout=dropout if num_layers>1 else 0.0)
        self.dropout = nn.Dropout(dropout)
        self.tie_weights = tie_weights
        if tie_weights:
            self.proj = nn.Linear(hidden_size, embedding_dim, bias=False) if hidden_size != embedding_dim else nn.Identity()
            self.decoder = nn.Linear(embedding_dim, vocab_size, bias=False)
            self.decoder.weight = self.emb.weight
        else:
            self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        e = self.emb(x)                 # (B, T, E)
        o, _ = self.rnn(e)              # (B, T, H)
        h = self.dropout(o[:, -1, :])   # (B, H)
        return self.decoder(self.proj(h)) if self.tie_weights else self.fc(h)  # (B, V)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_best_checkpoint(path):
    """Carga el checkpoint guardado en `path` y recrea el modelo preentrenado"""
    try:
        ckpt = torch.load(path, map_location="cpu")  # intenta modo seguro (weights_only=True)
    except Exception:
        ckpt = torch.load(path, map_location="cpu", weights_only=False)  # fallback
    cfg  = ckpt["config"]
    model = ChordLSTM(
        vocab_size=len(ckpt["stoi"]),
        embedding_dim=cfg["embedding_dim"],
        hidden_size=cfg["hidden_size"],
        num_layers=cfg["num_layers"],
        dropout=cfg["dropout"],
        tie_weights=cfg.get("tie_weights", False),
    )
    model.load_state_dict(ckpt["model_state"], strict=True)
    model.to(device).eval()
    return model, ckpt["stoi"], ckpt["itos"], cfg, ckpt.get("metrics_test", None)

model, stoi, itos, cfg, metrics = load_best_checkpoint("/content/lstm_rs_v3.pt")
print("Cargado. Métricas (test) guardadas:", metrics)


Cargado. Métricas (test) guardadas: {'loss': 2.252060778257626, 'ppl': 9.50730811622195, 'Top@1': 0.45036420395421434, 'Top@3': 0.6936524453818116, 'Top@5': 0.7761706556043317, 'MRR': 0.5956792587544246}


1.3. Definimos una función para obtener las predicciones con el modelo LSTM.
`predict_next_lstm()`

In [ ]:
#@title Definicion de `predict_next_lstm()` usando el chekpoint empaquetado
import torch.nn.functional as F

@torch.inference_mode()
def predict_next_lstm(model, stoi, itos, context_tokens, seq_len, k=5):
    unk_id = stoi.get("<unk>")
    if unk_id is None:
      raise KeyError("El vocabulario no contiene '<unk>'.")
    bos_id = stoi.get("<bos>")

    ids = [stoi.get(t, unk_id) for t in context_tokens]
    if len(ids) < seq_len and bos_id is not None:
        ids = [bos_id] * (seq_len - len(ids)) + ids
    else:
        ids = ids[-seq_len:]

    x = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)
    logits = model(x)                           # (1, V)
    probs  = F.softmax(logits[0], dim=-1)       # (V,)
    topk   = torch.topk(probs, k)
    return [(itos[i.item()], float(p.item())) for i,p in zip(topk.indices, topk.values)]


## 2) Cargamos el modelo kn

2.1. Importamos el modulo que contiene la clase del modelo Kneser-Ney desde GitHub.

In [ ]:
import urllib.request
import importlib.util

# Configuración para acceder al módulo que contiene la clase del modelo desde GitHub
USER = "antoniotrapote"
REPO = "chord-prediction-tfm"
BRANCH = "main"
PATH_IN_REPO = "anexos/models/kn_gs_v2/kn_model.py"
MODULE_PATH = f"https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}/{PATH_IN_REPO}"

urllib.request.urlretrieve(MODULE_PATH, "kn_model.py")
spec = importlib.util.spec_from_file_location("kn_model", "kn_model.py")
kn_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(kn_module)
KNInterpolatedNGram = kn_module.KNInterpolatedNGram

2.2. Cargamos el modelo preentrenado.

In [9]:
#@title Cargamos el modelo preentrenado
import joblib
import urllib.request

# Configuración para acceder al modelo entrenado desde GitHub
USER = "antoniotrapote"
REPO = "chord-prediction-tfm"
BRANCH = "main"
PATH_IN_REPO = "anexos/models/kn_gs_v2/best_kn_model.pkl"
TRAINED_MODEL_PATH = f"https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}/{PATH_IN_REPO}"

urllib.request.urlretrieve(TRAINED_MODEL_PATH, "best_kn_model.pkl")
best_kn = joblib.load("best_kn_model.pkl")

2.3. Definimos una función para obtener las predicciones con el modelo de Kneser-Ney.
`predict_next_kn()`

In [ ]:
#@title definimos `predict_next_kn()`
def predict_next_kn(model, context, k=5, exclude_special=True):
    """Devuelve las k mejores sugerencias (token, prob) dado el contexto."""
    ranking = model.predict_ranking(context)
    if exclude_special:
        ranking = [(w,p) for (w,p) in ranking if w not in {"<s>", "</s>", "<unk>"}]
    return ranking[:k]

## 3) Comparativa cualitativa de predicciones: KN vs LSTM

In [ ]:
import pandas as pd

K = 5                     # top-k
SEQ_LEN = cfg["seq_len"]  # del checkpoint LSTM

def count_hits(cands, acceptable):
    if not acceptable:
        return None
    s = set(acceptable)
    return sum(1 for c in cands if c in s)

def first_hit_rank(cands, acceptable):
    if not acceptable:
        return None
    s = set(acceptable)
    for i, c in enumerate(cands, start=1):
        if c in s:
            return i
    return None

def eval_case(context, acceptable=None, note=""):
    # KN
    kn_preds = predict_next_kn(best_kn, context, k=K)            # [('I', 0.21), ...]
    # LSTM
    lstm_preds = predict_next_lstm(model, stoi, itos, context, seq_len=SEQ_LEN, k=K)

    kn_top   = [c for c,_ in kn_preds]
    lstm_top = [c for c,_ in lstm_preds]

    row = {
        "context": " ".join(context),
        "note": note,
        "acc_targets": sorted(list(acceptable)) if acceptable else None,
        "kn_top": kn_top,
        "lstm_top": lstm_top,
    }

    # hit@k y rank (si hay acc_targets)
    row["KN_hit@k"]   = any(c in acceptable for c in kn_top)   if acceptable else None
    row["LSTM_hit@k"] = any(c in acceptable for c in lstm_top) if acceptable else None
    row["KN_rank"]    = first_hit_rank(kn_top, acceptable)
    row["LSTM_rank"]  = first_hit_rank(lstm_top, acceptable)

    # “plaus_sum” es simplemente el nº de aciertos dentro del top-k
    row["KN_plaus_sum"]   = count_hits(kn_top, acceptable)
    row["LSTM_plaus_sum"] = count_hits(lstm_top, acceptable)

    return row

# ---------- Suites de prueba (ROMANOS) ----------
SUITES = {
    "Modo mayor": [
        (["ii","V7"], {"I"}, "Cadencia auténtica"),
        (["I","V"], {"vi","IV","I"}, "Patrones pop"),
        (["I","IV"], {"I","V","ii"}, "Regreso a I o preparación a V"),
        (["IV","iv"], {"I"}, "Intercambio modal (iv→I)"),
        (["I","bVII"], {"I","IV","V"}, "Rock bVII"),
        (["I","V"], {"vi"}, "Cadencia punk I–V–vi–IV"),
        (["I","V","vi"], {"IV"}, "Cadencia punk I–V–vi–IV"),
    ],
    "Modo menor": [
        (["iiø","V7"], {"i"}, "iiø–V7–i"),
        (["i","iv"], {"V7", "v"}, "Preparar dominante"),
        (["VI","iiø"], {"V7"}, "VI–iiø–V7"),
    ],
    "Secundarios y sustitutos": [
        (["i","V/IV"], {"iv"}, "V/IV→iv"),
        (["I","V/IV"], {"IV"}, "V/IV→IV"),
        (["I","Vsub/V"], {"V7","V"}, "Sust. tritono a V"),
        (["I","biio"], {"ii"}, "bii°→ii (vecindad cromática)"),
        (["I","V/ii"], {"ii"}, "Cadena de dominantes"),
    ],
    "Cold-start y rarezas": [
        ([], {"I","i","ii","iiø"}, "Comienzo sin contexto"),
        (["i"], {"iiø","iv","V7","vi"}, "Arranque menor"),
        (["I"], {"ii","IV","V7","VI"}, "Arranque mayor"),
    ],
}

# ---------- Ejecutar ----------
rows = []
for suite, cases in SUITES.items():
    for ctx, acceptable, note in cases:
        rows.append({"suite": suite, **eval_case(ctx, acceptable, note)})

df = pd.DataFrame(rows)

def summarize(group):
    return pd.Series({
        "cases": len(group),
        "KN_hits": group["KN_hit@k"].fillna(False).sum() if "KN_hit@k" in group else None,
        "LSTM_hits": group["LSTM_hit@k"].fillna(False).sum() if "LSTM_hit@k" in group else None,
        "KN_mean_rank": group["KN_rank"].dropna().mean() if group["KN_rank"].notna().any() else None,
        "LSTM_mean_rank": group["LSTM_rank"].dropna().mean() if group["LSTM_rank"].notna().any() else None,
        # promedio de aciertos dentro del top-k
        "KN_plaus_sum_avg": group["KN_plaus_sum"].dropna().mean(),
        "LSTM_plaus_sum_avg": group["LSTM_plaus_sum"].dropna().mean(),
    })

summary = df.groupby("suite").apply(summarize, include_groups=False).reset_index()

global_scores = pd.DataFrame([{
    "KN_plaus_avg": df["KN_plaus_sum"].dropna().mean(),
    "LSTM_plaus_avg": df["LSTM_plaus_sum"].dropna().mean(),
    "KN_hit_rate": df["KN_hit@k"].fillna(False).mean(),
    "LSTM_hit_rate": df["LSTM_hit@k"].fillna(False).mean(),
}]).round(3)

print("=== RESUMEN POR SUITE ===")
display(summary.round(3))
print("\n=== SCORES GLOBALES ===")
display(global_scores)

cols_show = ["suite","context","note","acc_targets",
             "kn_top","lstm_top","KN_rank","LSTM_rank",
             "KN_plaus_sum","LSTM_plaus_sum"]
display(df[cols_show])


=== RESUMEN POR SUITE ===


,suite,cases,KN_hits,LSTM_hits,KN_mean_rank,LSTM_mean_rank,KN_plaus_sum_avg,LSTM_plaus_sum_avg
0,Cold-start y rarezas,3.0,3.0,0.0,1.667,NaN,3.000,0.000
1,Modo mayor,7.0,6.0,4.0,1.667,2.25,1.429,0.714
2,Modo menor,3.0,3.0,2.0,1.000,1.50,1.000,0.667
3,Secundarios y sustitutos,5.0,5.0,4.0,1.400,1.00,1.200,1.000



=== SCORES GLOBALES ===


,KN_plaus_avg,LSTM_plaus_avg,KN_hit_rate,LSTM_hit_rate
0,1.556,0.667,0.944,0.556


,suite,context,note,acc_targets,kn_top,lstm_top,KN_rank,LSTM_rank,KN_plaus_sum,LSTM_plaus_sum
0,Modo mayor,ii V7,Cadencia auténtica,[I],"[I, ii, iii, i, V7]","[V7, I, bii, Vsub/VII, #IV7]",1.0,2.0,1,1
1,Modo mayor,I V,Patrones pop,"[I, IV, vi]","[V, IV, I, vi, v]","[V, I, bii, II, vo]",2.0,2.0,3,1
2,Modo mayor,I IV,Regreso a I o preparación a V,"[I, V, ii]","[I, iii, V7, V, IV]","[IV, VII, iii, I, bIII]",1.0,4.0,2,1
3,Modo mayor,IV iv,Intercambio modal (iv→I),[I],"[I, bVII7, iii, iv, V]","[iii, iv, V7, bIII7, VII]",1.0,NaN,1,0
4,Modo mayor,I bVII,Rock bVII,"[I, IV, V]","[I, IV, bVII, V/IV, V7]","[I, bii, IV, V7, vii]",1.0,1.0,2,2
5,Modo mayor,I V,Cadencia punk I–V–vi–IV,[vi],"[V, IV, I, vi, v]","[V, I, bii, II, vo]",4.0,NaN,1,0
6,Modo mayor,I V vi,Cadencia punk I–V–vi–IV,[IV],"[II7, V, V/V, Vsub/II, vi]","[II, V/V, vii, V, II7]",NaN,NaN,0,0
7,Modo menor,iiø V7,iiø–V7–i,[i],"[i, I, V7, iiø, VI]","[V7, bii, vo, #IV7, I]",1.0,NaN,1,0
8,Modo menor,i iv,Preparar dominante,"[V7, v]","[V7, i, iv, V/III, bVII7]","[iv, V7, bIII7, i, iii]",1.0,2.0,1,1
9,Modo menor,VI iiø,VI–iiø–V7,[V7],"[V7, iiø, i, I, bVI7]","[V7, vo, bii, iiø, bII7]",1.0,1.0,1,1


## 4) Conclusiones 

### Justificación de la elección del modelo Kneser-Ney

A pesar de que el LSTM ofrecía métricas ligeramente mejores en validación (Top@k, MRR), en las pruebas cualitativas con progresiones armónicas reales se observa que el modelo KN interpolado:

- Tiene una tasa de acierto mucho más alta (94% vs 56%).

- Ofrece sugerencias más plausibles y consistentes en todos los contextos (cadencias, arranques, dominantes secundarios).

- Es estable, explicable y determinista, lo que lo hace más adecuado para un sistema productivo.